In [ ]:
import sympy as sp
from sympy import *

sp.init_session()
sp.init_printing()

### Selection of algorithm:
- `divE_cleaning` (bool: `True`, `False`): "$\text{div}(\boldsymbol{E})$ cleaning" using the additional scalar field $F$;

- `divB_cleaning` (bool: `True`, `False`): "$\text{div}(\boldsymbol{B})$ cleaning" using the additional scalar field $G$;

- `J_in_time` (str: `'constant'`, `'linear'`, `'quadratic'`): $\boldsymbol{J}$ constant, linear, or quadratic in time;

- `rho_in_time` (str: `'constant'`, `'linear'`, `'quadratic'`): $\rho$ constant, linear, or quadratic in time.

In the notebook, the constant, linear, and quadratic coefficients of $\boldsymbol{J}$ and $\rho$ are denoted with the suffixes `_c0`, `_c1`, `_c2`, respectively. However, the corresponding coefficients in the displayed equations are denoted with the prefixes $\gamma$, $\beta$, and $\alpha$, respectively. For example, if $\rho$ is quadratic in time, it will be denoted as `rho = rho_c0 + rho_c1*(t-tn) + rho_c2*(t-tn)**2` in the notebook and $\rho(t) = \gamma_{\rho} + \beta_{\rho} (t-t_n) + \alpha_{\rho} (t-t_n)^2$ in the displayed equations.

In [ ]:
divE_cleaning = True
divB_cleaning = True
J_in_time = 'constant'
rho_in_time = 'constant'

### Auxiliary functions:

In [ ]:
def check_diag(W, D, P, invP):
    """
    Check diagonalization of W as P*D*P**(-1).
    """
    Wd = P * D * invP
    for i in range(Wd.shape[0]):
        for j in range(Wd.shape[1]):
            Wd[i,j] = Wd[i,j].expand().simplify()
            diff = W[i,j] - Wd[i,j]
            diff = diff.expand().simplify()
            assert (diff == 0), f'Diagonalization failed: W[{i},{j}] - Wd[{i},{j}] = {diff} is not zero'

def simple_mat(W):
    """
    Simplify matrix W.
    """
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            W[i,j] = W[i,j].expand().simplify()

### Definition of symbols:

In [ ]:
# Define dimensionality parameter used throughout the notebook
dim = 6
if divE_cleaning:
    dim += 1
if divB_cleaning:
    dim += 1

# Define symbols for physical constants
c = sp.symbols(r'c', real=True, positive=True)
mu0 = sp.symbols(r'\mu_0', real=True, positive=True)

# Define symbols for time variables
# (s is auxiliary variable used in integral over time)
s = sp.symbols(r's', real=True, positive=True)
t = sp.symbols(r't', real=True, positive=True)
tn = sp.symbols(r't_n', real=True, positive=True)
dt = sp.symbols(r'\Delta{t}', real=True, positive=True)

# The assumption that kx, ky and kz are positive is general enough
# and makes it easier for SymPy to perform some of the calculations
kx = sp.symbols(r'k_x', real=True, positive=True)
ky = sp.symbols(r'k_y', real=True, positive=True)
kz = sp.symbols(r'k_z', real=True, positive=True)

# Define symbols for the Cartesian components of the electric field
Ex = sp.symbols(r'E^x')
Ey = sp.symbols(r'E^y')
Ez = sp.symbols(r'E^z')
E = Matrix([[Ex], [Ey], [Ez]])

# Define symbols for the Cartesian components of the magnetic field
Bx = sp.symbols(r'B^x')
By = sp.symbols(r'B^y')
Bz = sp.symbols(r'B^z')
B = Matrix([[Bx], [By], [Bz]])

# Define symbol for the scalar field F used with div(E) cleaning
if divE_cleaning:
    F = sp.symbols(r'F')

# Define symbol for the scalar field G used with div(B) cleaning
if divB_cleaning:
    G = sp.symbols(r'G')

### First-order ODEs for $\boldsymbol{E}$,  $\boldsymbol{B}$, $F$ and $G$:

In [ ]:
# Define first-order time derivatives of the electric field
dEx_dt = I*c**2*(ky*Bz-kz*By) 
dEy_dt = I*c**2*(kz*Bx-kx*Bz)
dEz_dt = I*c**2*(kx*By-ky*Bx)

# Define first-order time derivatives of the magnetic field
dBx_dt = -I*(ky*Ez-kz*Ey)
dBy_dt = -I*(kz*Ex-kx*Ez)
dBz_dt = -I*(kx*Ey-ky*Ex)

# Define first-order time derivative of the scalar field F used with div(E) cleaning,
# and related additional terms in the first-order time derivative of the electric field
if divE_cleaning:
    dEx_dt += I*c**2*F*kx 
    dEy_dt += I*c**2*F*ky
    dEz_dt += I*c**2*F*kz
    dF_dt = I*(kx*Ex+ky*Ey+kz*Ez)

# Define first-order time derivative of the scalar field G used with div(B) cleaning,
# and related additional terms in the first-order time derivative of the magnetic field
if divB_cleaning:
    dBx_dt += I*c**2*G*kx
    dBy_dt += I*c**2*G*ky
    dBz_dt += I*c**2*G*kz
    dG_dt = I*(kx*Bx+ky*By+kz*Bz)

# Define array of first-order time derivatives of the electric and magnetic fields
dE_dt = Matrix([[dEx_dt], [dEy_dt], [dEz_dt]])
dB_dt = Matrix([[dBx_dt], [dBy_dt], [dBz_dt]])

### Linear system of ODEs for $\boldsymbol{E}$, $\boldsymbol{B}$, $F$ and $G$:
$$
\frac{\partial}{\partial t}
\begin{bmatrix}
\boldsymbol{E} \\
\boldsymbol{B} \\
F \\
G
\end{bmatrix}
= M
\begin{bmatrix}
\boldsymbol{E} \\
\boldsymbol{B} \\
F \\
G
\end{bmatrix}
-\mu_0 c^2 
\begin{bmatrix}
\boldsymbol{J} \\
\boldsymbol{0} \\
\rho \\
 0
\end{bmatrix}
$$

In [ ]:
# Define array of all fields
fields_list = [Ex, Ey, Ez, Bx, By, Bz]
if divE_cleaning:
    fields_list.append(F)
if divB_cleaning:
    fields_list.append(G)
EBFG = zeros(dim, 1)
for i in range(EBFG.shape[0]):
    EBFG[i] = fields_list[i]

# Define array of first-order time derivatives of all fields
fields_list = [dEx_dt, dEy_dt, dEz_dt, dBx_dt, dBy_dt, dBz_dt]
if divE_cleaning:
    fields_list.append(dF_dt)
if divB_cleaning:
    fields_list.append(dG_dt)
dEBFG_dt = zeros(dim, 1)
for i in range(dEBFG_dt.shape[0]):
    dEBFG_dt[i] = fields_list[i]
dEBFG_dt = dEBFG_dt.expand()

# Define matrix M representing the linear system of ODEs
M = zeros(dim)
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        M[i,j] = dEBFG_dt[i].coeff(EBFG[j], 1)
print(r'M = ')
display(M)

### Solution of linear system of ODEs for $\boldsymbol{E}$, $\boldsymbol{B}$, $F$ and $G$:

The solution of the linear system of ODEs above is given by the superposition of the **general solution of the homogeneous system** (denoted with the subscript "h"),

$$
\begin{bmatrix}
\boldsymbol{E}_h(t) \\
\boldsymbol{B}_h(t) \\
F_h(t) \\
G_h(t)
\end{bmatrix}
= e^{M (t-t_n)}
\begin{bmatrix}
\boldsymbol{E}(t_n) \\
\boldsymbol{B}(t_n) \\
F(t_n) \\
G(t_n)
\end{bmatrix} \,,
$$

and the **particular solution of the non-homogeneous system** (denoted with the subscript "nh"),

$$
\begin{bmatrix}
\boldsymbol{E}_{nh}(t) \\
\boldsymbol{B}_{nh}(t) \\
F_{nh}(t) \\
G_{nh}(t)
\end{bmatrix}
= -\mu_0 c^2 e^{M t} \left(\int_{t_n}^t e^{-M s}
\begin{bmatrix}
\boldsymbol{J} \\
\boldsymbol{0} \\
\rho \\
0
\end{bmatrix}
ds\right) \,.
$$

### Diagonalization of $M = P D P^{-1}$:

In [ ]:
%%time

# Compute matrices of eigenvectors and eigenvalues for diagonalization of M
P, D = M.diagonalize()
invP = P**(-1)
expD = exp(D)
check_diag(M, D, P, invP)
print('P = ')
display(P)
print('D = ')
display(D)

### Diagonalization of $W_1 = M (t-t_n) = P_1 D_1 P_1^{-1}$:

In [ ]:
%%time

# Compute matrices of eigenvectors and eigenvalues for diagonalization of W1
P1 = P
D1 = D * (t-tn)
invP1 = invP
expD1 = exp(D1)

### Diagonalization of $W_2 = M t = P_2 D_2 P_2^{-1}$:

In [ ]:
%%time

# Compute matrices of eigenvectors and eigenvalues for diagonalization of W2
P2 = P
D2 = D * t
invP2 = invP
expD2 = exp(D2)

### Diagonalization of $W_3 = -M s = P_3 D_3 P_3^{-1}$:

In [ ]:
%%time

# Compute matrices of eigenvectors and eigenvalues for diagonalization of W3
P3 = P
D3 = (-1) * D * s
invP3 = invP
expD3 = exp(D3)

### General solution (homogeneous system):

$$
\begin{bmatrix}
\boldsymbol{E}_h(t) \\
\boldsymbol{B}_h(t) \\
F_h(t) \\
G_h(t)
\end{bmatrix}
= e^{M (t-t_n)}
\begin{bmatrix}
\boldsymbol{E}(t_n) \\
\boldsymbol{B}(t_n) \\
F(t_n) \\
G(t_n)
\end{bmatrix}
$$

In [ ]:
%%time

# Compute exp(W1) = exp(M*(t-tn))
expW1 = P1 * expD1 * invP1

# Compute general solution at time t = tn+dt
EBFG_h = expW1 * EBFG 
EBFG_h_new = EBFG_h.subs(t, tn+dt)

### Definition of $\boldsymbol{J}$ and $\rho$:

In [ ]:
# Define J
Jx_c0 = sp.symbols(r'\gamma_{J_x}', real=True)
Jy_c0 = sp.symbols(r'\gamma_{J_y}', real=True)
Jz_c0 = sp.symbols(r'\gamma_{J_z}', real=True)
Jx = Jx_c0
Jy = Jy_c0
Jz = Jz_c0
if J_in_time == 'linear':
    Jx_c1 = sp.symbols(r'\beta_{J_x}', real=True)
    Jy_c1 = sp.symbols(r'\beta_{J_y}', real=True)
    Jz_c1 = sp.symbols(r'\beta_{J_z}', real=True)
    Jx += Jx_c1*(s-tn)
    Jy += Jy_c1*(s-tn)
    Jz += Jz_c1*(s-tn)
if J_in_time == 'quadratic':
    Jx_c1 = sp.symbols(r'\beta_{J_x}', real=True)
    Jy_c1 = sp.symbols(r'\beta_{J_y}', real=True)
    Jz_c1 = sp.symbols(r'\beta_{J_z}', real=True)
    Jx_c2 = sp.symbols(r'\alpha_{J_x}', real=True)
    Jy_c2 = sp.symbols(r'\alpha_{J_y}', real=True)
    Jz_c2 = sp.symbols(r'\alpha_{J_z}', real=True)
    Jx += Jx_c1*(s-tn) + Jx_c2*(s-tn)**2
    Jy += Jy_c1*(s-tn) + Jy_c2*(s-tn)**2
    Jz += Jz_c1*(s-tn) + Jz_c2*(s-tn)**2

# Define rho
if divE_cleaning:
    rho_c0 = sp.symbols(r'\gamma_{\rho}', real=True)
    rho = rho_c0
    if rho_in_time == 'linear':
        rho_c1 = sp.symbols(r'\beta_{\rho}', real=True)
        rho += rho_c1*(s-tn)
    if rho_in_time == 'quadratic':
        rho_c1 = sp.symbols(r'\beta_{\rho}', real=True)
        rho_c2 = sp.symbols(r'\alpha_{\rho}', real=True)
        rho += rho_c1*(s-tn) + rho_c2*(s-tn)**2

### Particular solution (non-homogeneous system):

$$
\begin{bmatrix}
\boldsymbol{E}_{nh}(t) \\
\boldsymbol{B}_{nh}(t) \\
F_{nh}(t) \\
G_{nh}(t)
\end{bmatrix}
= -\mu_0 c^2 e^{M t} \left(\int_{t_n}^t e^{-M s}
\begin{bmatrix}
\boldsymbol{J} \\
\boldsymbol{0} \\
\rho \\
0
\end{bmatrix}
ds\right)
$$

In [ ]:
%%time 

# Define array of source terms
fields_list = [Jx, Jy, Jz, 0, 0, 0]
if divE_cleaning:
    fields_list.append(rho)
if divB_cleaning:
    fields_list.append(0)
S = zeros(dim, 1)
for i in range(S.shape[0]):
    S[i] = -mu0*c**2 * fields_list[i]

# Compute integral of exp(W3)*S over s (assuming |k| is not zero)
integral = zeros(dim, 1)
tmp = expD3 * invP3 * S
simple_mat(tmp)
for i in range(dim):
    r = integrate(tmp[i], (s, tn, t))
    integral[i] = r
    
# Compute particular solution at time t = tn+dt
tmp = invP2 * P3
simple_mat(tmp)
EBFG_nh = P2 * expD2 * tmp * integral
EBFG_nh_new = EBFG_nh.subs(t, tn+dt)

### Verification of the solution:

In [ ]:
%%time

for i in range(EBFG.shape[0]):
    lhs = dEBFG_dt[i] + S[i]
    lhs = lhs.subs(s, tn) # sources were written as functions of s
    rhs = (EBFG_h[i] + EBFG_nh[i]).diff(t)
    rhs = rhs.subs(t, tn) # results were written as functions of t
    rhs = rhs.simplify()
    diff = lhs - rhs
    diff = diff.simplify()
    assert (diff == 0), f'Integration of linear system of ODEs failed'

### Coefficients of PSATD equations (homogeneous system):

In [ ]:
%%time

L = ['Ex', 'Ey', 'Ez', 'Bx', 'By', 'Bz']
R = ['Ex', 'Ey', 'Ez', 'Bx', 'By', 'Bz']
if divE_cleaning:
    L.append('F')
    R.append('F')
if divB_cleaning:
    L.append('G')
    R.append('G')

# Compute individual coefficients in the update equations
coeff_h = dict()
for i in range(dim):
    for j in range(dim):
        key = (L[i], R[j])
        coeff_h[key] = EBFG_h_new[i].coeff(EBFG[j], 1).expand().simplify().rewrite(cos).trigsimp().simplify()
        print(f'Coefficient of {L[i]} with respect to {R[j]}:')
        display(coeff_h[key])

### Coefficients of PSATD equations (non-homogeneous system):

In [ ]:
%%time

L = ['Ex', 'Ey', 'Ez', 'Bx', 'By', 'Bz']
R = ['Jx_c0', 'Jy_c0', 'Jz_c0']
if J_in_time == 'linear':
    R.append('Jx_c1')
    R.append('Jy_c1')
    R.append('Jz_c1')
if J_in_time == 'quadratic':
    R.append('Jx_c1')
    R.append('Jy_c1')
    R.append('Jz_c1')
    R.append('Jx_c2')
    R.append('Jy_c2')
    R.append('Jz_c2')
if divE_cleaning:
    L.append('F')
    R.append('rho_c0')
    if rho_in_time == 'linear':
        R.append('rho_c1')
    if rho_in_time == 'quadratic':
        R.append('rho_c1')
        R.append('rho_c2')
if divB_cleaning:
    L.append('G')

cs = [Jx_c0, Jy_c0, Jz_c0]
if J_in_time == 'linear':
    cs.append(Jx_c1)
    cs.append(Jy_c1)
    cs.append(Jz_c1)
if J_in_time == 'quadratic':
    cs.append(Jx_c1)
    cs.append(Jy_c1)
    cs.append(Jz_c1)
    cs.append(Jx_c2)
    cs.append(Jy_c2)
    cs.append(Jz_c2)
if divE_cleaning:
    cs.append(rho_c0)
    if rho_in_time == 'linear':
        cs.append(rho_c1)
    if rho_in_time == 'quadratic':
        cs.append(rho_c1)
        cs.append(rho_c2)
    
# Compute individual coefficients in the update equation
coeff_nh = dict()
for i in range(len(L)):
    for j in range(len(R)):
        key = (L[i], R[j])
        coeff_nh[key] = EBFG_nh_new[i].expand().coeff(cs[j], 1).expand().simplify().rewrite(cos).trigsimp().simplify()
        print(f'Coefficient of {L[i]} with respect to {R[j]}:')
        display(coeff_nh[key])

### Coefficients of PSATD equations:

Display the coefficients of the update equations one by one. For example, `coeff_h[('Ex', 'By')]` displays the coefficient of $E^x$ with respect to $B^y$ (resulting from the solution of the homogeneous system, hence the `_h` suffix), while `coeff_nh[('Ex', 'Jx_c0')]` displays the coefficient of $E^x$ with respect to $\gamma_{J_x}$ (resulting from the solution of the non-homogeneous system, hence the `_nh` suffix). Note that $\gamma_{J_x}$ is denoted as `Jx_c0` in the notebook, as described in the beginning.

In [ ]:
coeff_h[('Ex', 'By')]

In [ ]:
coeff_nh[('Ex', 'Jx_c0')]